<a href="https://colab.research.google.com/github/AvRob0/Avrob0.github.io/blob/main/RobinsonAveryFinalProjectEnd_To_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imported librarys needed for End-to-end implementation of a Data Science Project
import pandas as pd
import numpy as np
# Loaded the dataset into the hotel_df variable
# and selected all the columns to see how many there are
hotel_df = pd.read_csv('hotel_bookings.csv')
hotel_df.columns

In [ ]:
hotel_df.info()
# Getting information about the dataset
# Getting the info of the dataset gives me the Range of the Index which is 119390
# It also gives me the total number of coloumns that are in the dataset and gives me all the columns names
# With their non-Null count and datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [ ]:
hotel_df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [ ]:
hotel_df.shape

(119390, 32)

In [ ]:
for i in hotel_df.columns:
    #print(i)
    #print(hotel_df[i].unique())
    if hotel_df[i].isna().sum() != 0:
      print('null values in', i, 'column', hotel_df[i].isna().sum())

null values in children column 4
null values in country column 488
null values in agent column 16340
null values in company column 112593


In [ ]:
# drop the company column since most of the values are 0
hotel_df = hotel_df.drop(columns = 'company')

# drops 4 rows all containing 0 value in the children column
hotel_df = hotel_df.dropna(subset=['children'])
#hotel_df
# filling the missing values in country column with the frequent mode value
hotel_df['country'].fillna(hotel_df['country'].mode()[0], inplace=True)

# then fill all the missing values in the agent column and use the value used for agent id.
hotel_df['agent'].fillna(0, inplace=True)
